<h1 id="tocheading">Table of Contents</h1>
<div id="toc"></div>

In [1]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

<IPython.core.display.Javascript object>

### 262. Trips and Users

```mysql
SELECT request_at AS Day, round(sum(cancelled)/count(client_id), 2) AS "Cancellation Rate"
FROM (
    SELECT client_id, request_at,
        CASE WHEN status='completed' THEN 0
        ELSE 1 END as cancelled
    FROM trips
    JOIN users ON trips.client_id=users.users_id AND users.banned='No'
    WHERE request_at BETWEEN "2013-10-01" AND "2013-10-03"
        AND role='client') A
GROUP BY request_at

/*SELECT
	Trips.Request_at AS Day,
	CONVERT (
		SUM( CASE WHEN Trips.STATUS LIKE 'cancelled_by_%' THEN 1 ELSE 0 END ) / COUNT( * ),
		DECIMAL ( 10, 2 ) 
	) AS `Cancellation Rate`
FROM
	Trips 
WHERE
	Trips.Client_Id NOT IN ( SELECT Users_Id FROM Users WHERE Banned = 'Yes' AND Role = 'client' ) 
	AND Trips.Driver_Id NOT IN ( SELECT Users_Id FROM Users WHERE Banned = 'Yes' AND Role = 'driver' )
    AND Trips.Request_at BETWEEN '2013-10-01' AND '2013-10-03'
GROUP BY
	Request_at*/
```

### 511. Game Play Analysis I

```mysql
SELECT player_id, MIN(event_date) AS first_login
FROM activity
GROUP BY player_id
```

### 512. Game Play Analysis II

```mysql
-- MYSQL
/*SELECT  a.player_id, device_id
FROM activity a
JOIN (
    SELECT player_id, MIN(event_date) as first_login
    FROM activity
    GROUP BY player_id
) b
ON a.player_id=b.player_id AND event_date=first_login*/

SELECT  player_id, device_id
FROM activity
WHERE (player_id, event_date) IN (
    SELECT player_id, MIN(event_date) as login
    FROM activity
    GROUP BY player_id
)

-- SQL server
SELECT  player_id, device_id
FROM (
    SELECT player_id, device_id,
    RANK() OVER (PARTITION BY player_id ORDER BY event_date) as login_no
    FROM activity
) x
WHERE login_no=1
```

### 534. Game Play Analysis III

```mysql
-- MYSQL
/*SELECT a.player_id, a.event_date, sum(b.games_played) AS games_played_so_far 
FROM activity a
JOIN activity b
WHERE a.event_date>=b.event_date AND a.player_id=b.player_id
GROUP BY a.player_id, a.event_date*/

select a1.player_id, a1.event_date, sum(a2.games_played) games_played_so_far
from Activity a1, Activity a2
where a1.event_date >= a2.event_date
and a1.player_id = a2.player_id
group by a1.player_id, a1.event_date

-- SQL server
SELECT player_id, event_date,
    SUM(games_played) OVER (PARTITION BY player_id ORDER BY event_date) AS games_played_so_far
FROM activity
```

### 550. Game Play Analysis IV

```mysql
-- MYSQL
/* SELECT ROUND(COUNT(a.player_id)/(SELECT COUNT(DISTINCT player_id) FROM activity), 2) AS fraction
FROM activity a
JOIN (
    SELECT player_id, MIN(event_date) AS first_login
    FROM activity
    GROUP BY player_id) b
ON a.player_id=b.player_id AND DATEDIFF(a.event_date, first_login)=1 */

SELECT ROUND(SUM(
    CASE WHEN a.first_login +1 = a2.event_date THEN 1 ELSE 0 END)
             / COUNT(DISTINCT a.player_id),2) AS fraction
FROM (
    SELECT player_id, MIN(event_date) AS first_login
    FROM activity
    GROUP BY player_id) a
JOIN activity a2
ON a.player_id=a2.player_id

-- SQL server
SELECT ROUND( COUNT(DISTINCT player_id)*1.0 / (SELECT COUNT(DISTINCT player_id) FROM activity), 2) AS fraction
FROM (
    SELECT player_id, event_date,
    DATEDIFF(day, LAG(event_date, 1) OVER (PARTITION BY player_id ORDER BY event_date), event_date) AS diff,
    RANK() OVER (PARTITION BY player_id ORDER BY event_date) AS rnk
    FROM activity
    ) x
WHERE diff=1 AND rnk=2
```

### 569. Median Employee Salary

```mysql
-- MYSQL
SELECT
    Employee.Id, Employee.Company, Employee.Salary
FROM
    Employee,
    Employee alias
WHERE
    Employee.Company = alias.Company
GROUP BY Employee.Company , Employee.Salary
HAVING SUM(
    CASE WHEN Employee.Salary = alias.Salary THEN 1
    ELSE 0 END)
    >= ABS(SUM(SIGN(Employee.Salary - alias.Salary)))
ORDER BY Employee.Id

-- SQL server
SELECT id, company, salary
FROM (
    SELECT id, company, salary,
        COUNT(*) OVER (PARTITION BY company) as num_employee,
        ROW_NUMBER() OVER (PARTITION BY company ORDER BY salary) as seq_employee
    FROM employee) AS x
WHERE seq_employee IN (CEILING((num_employee+1)/2.0), FLOOR((num_employee+1)/2.0))
```

### 570. Managers with at Least 5 Direct Reports

```mysql
SELECT name
FROM employee
WHERE Id IN (
    SELECT managerId
    FROM employee
    GROUP BY managerId
    HAVING count(*)>=5)
```

### 571. Find Median Given Frequency of Numbers

```mysql
-- MYSQL
SELECT AVG(median) AS 'median'
FROM (
    SELECT a.Number as median, a.Frequency
    FROM Numbers a, Numbers b
    GROUP BY a.Number
    HAVING
        a.Frequency >= abs(Sum(
            CASE WHEN a.Number-b.Number > 0 THEN 1 * b.Frequency
                WHEN a.Number=b.Number THEN 0
                ELSE -1 * b.Frequency
            END))
) X

-- SQL server
with temp as (
    select sum(frequency) over(order by number) as cum_total,
    (sum(frequency) over(order by number) - frequency) as cum_excluded,
    sum(frequency) over () total,
    number, frequency
    from Numbers)

select avg(cast(number as decimal(6,2))) as median
from temp
where (total/2.0) between cum_excluded and cum_total

/*
SELECT sum(number)/cast(count(number) AS float) as median
FROM (SELECT number, frequency,
    sum(frequency) OVER (ORDER BY number ROWS BETWEEN unbounded preceding AND current ROW) AS CUM_NUM_high,
    sum(frequency) OVER (ORDER BY number ROWS BETWEEN unbounded preceding AND 1 preceding) AS CUM_NUM_low
FROM Numbers) AS X
WHERE (SELECT sum(frequency)/2.0 FROM numbers) BETWEEN coalesce(cum_num_low, 0) AND cum_num_high
*/
```

### 574. Winning Candidate

```mysql
-- MYSQL
SELECT name
FROM candidate
JOIN vote
ON vote.candidateid=candidate.id
GROUP BY candidate.id
HAVING  count(vote.id)=(
    SELECT MAX(votes)
    FROM (
        SELECT count(*) AS votes
        FROM candidate
        RIGHT JOIN vote
        ON vote.candidateid=candidate.id
        GROUP BY candidate.id
        ) X
    )

-- SQL server
SELECT C.Name
FROM Candidate C
JOIN (
    SELECT CandidateId, RANK() OVER (ORDER BY COUNT(CandidateId) desc) AS rnk
    FROM Vote
    GROUP BY CandidateId) Voting
    ON C.id = Voting.CandidateId
    WHERE Voting.rnk=1
```

### 577. Employee Bonus

```mysql
SELECT name, bonus
FROM employee as a
LEFT JOIN bonus as b
ON a.empid=b.empid 
WHERE b.bonus<1000 OR b.bonus IS NULL
```

### 578. Get Highest Answer Rate Question

```mysql
SELECT question_id as survey_log
FROM survey_log
GROUP BY question_id
HAVING (SUM(IF(action='answer',1,0))/COUNT(action))=(
    SELECT MAX(rate)
    FROM (
        SELECT SUM(IF(action='answer',1,0))/COUNT(action) as rate
        FROM survey_log
        GROUP BY question_id) X
    )

/*SELECT question_id as survey_log
FROM
(
    SELECT question_id,
         SUM(case when action="answer" THEN 1 ELSE 0 END) as num_answer,
        SUM(case when action="show" THEN 1 ELSE 0 END) as num_show 
    FROM survey_log
    GROUP BY question_id
) as tbl
ORDER BY (num_answer / (num_show+num_answer)) DESC
LIMIT 1*/
```

### 579. Find Cumulative Salary of an Employee

```mysql
-- MYSQL
SELECT t1.id, t1.month, t1.salary
FROM (
    SELECT a1.id, b1.month, sum(a1.salary) as salary
    FROM employee a1, employee b1
    where a1.id=b1.id AND a1.month between b1.month-2 and b1.month
    group by a1.id, b1.month) t1
JOIN (
    SELECT id, max(month) as month
    FROM (SELECT a.id, b.month, sum(a.salary) as salary
    FROM employee a, employee b
    where a.id=b.id AND a.month between b.month-2 and b.month
    group by a.id, b.month) X
    group by id) t2
ON t1.id=t2.id AND t1.month!=t2.month
ORDER BY t1.id, t1.month DESC

-- SQL server
SELECT id, month, salary
FROM (
    SELECT id, month,
    SUM(Salary) OVER (PARTITION BY id ORDER BY month ROWS BETWEEN 2 PRECEDING AND CURRENT ROW) AS salary,
    RANK() OVER (PARTITION BY id ORDER BY month DESC) as month_rank
    FROM employee
) AS X
WHERE month_rank!=1
ORDER BY id, month DESC

/* SELECT A.ID,A.MONTH,
    SUM(SALARY) over (PARTITION BY A.ID order by month ROWS BETWEEN 2 PRECEDING AND CURRENT ROW ) salary
FROM EMPLOYEE A
WHERE (A.ID, A.MONTH)
    NOT IN (SELECT ID,MAX(MONTH) FROM EMPLOYEE_INPUT GROUP BY ID)
ORDER BY A.ID,A.MONTH DESC*/
```

### 580. Count Student Number in Departments

```mysql
SELECT dept_name, COUNT(student_name) as student_number
FROM department d
LEFT JOIN student s
ON d.dept_id=s.dept_id
GROUP BY d.dept_name
ORDER BY student_number DESC, d.dept_name
```